# TP MLFLOW
Florent Jakubowski

## A. Prise en main

1.Installez le package mlflow avec python dans un environnement virtuel.

2.Dans un terminal lancez un serveur mlflow. Aller voir dans votre navigateur, sur le port correspondant, l'ui de mlflow.

mlflow server --host 0.0.0.0

In [ ]:
!mlflow server --host 0.0.0.0

Lorsque vous lancez mflow sans option par défaut mlflow va stocker toute la donnée dont il a besoin sur votre file system. Vous aurez notamment un dossier mlruns qui se créra par défaut.

3.Dans un notebook, utilisez le package mlflow pour vous connecter au serveur mlflow que vous avez lancé. Utilisez la bonne fonction pour paramétrer l'adresse du serveur. 

In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

Attention ! A chaque fois que vous effectuerez une opération sur mlflow dans une autre cellule de votre notebook vous devrez vérifier avant que vous pointez bien sur le bon serveur mlflow. Il existe aussi une fonction pour connaître quelle adresse de serveur a été enregistrée. 

In [23]:
mlflow.get_tracking_uri()

'http://localhost:5000'

Créez une experiment via votre notebook ou avec via l'ui. 
Une experiment, ou une expérience en français, est un ensemble de run que vous avez effectué. Le but est de trouver in fine les meilleurs paramètres, modèles ou hyperparamètres pour votre besoin.

In [14]:
try : 
    experiment_id = mlflow.create_experiment("My Experiment")
    print("Experiment created with ID: ", experiment_id)
except:
    print("Experiment already exists")

Experiment already exists


Pareillement à l'adresse du serveur mlflow à chaque fois que vous exécuterez un run dans une cellule vous devrez définir l'experiment sur laquelle vous voulez envoyer votre run. Cherchez dans la documentation la fonction permettant de faire cela.

In [4]:
mlflow.set_experiment("My Experiment")

<Experiment: artifact_location='mlflow-artifacts:/636497380083101659', creation_time=1700040769157, experiment_id='636497380083101659', last_update_time=1700040769157, lifecycle_stage='active', name='My Experiment', tags={}>

Nous vous proposons de créer une fonction `configure_experiment` permettant de créer une expérience ou de définir l'expérience si elle existe déjà.

In [81]:
def config_exp(name):
    # Check if the experiment exists
    experiment = mlflow.get_experiment_by_name(name)
    
    if experiment is None:
        # If the experiment does not exist, create it
        experiment_id = mlflow.create_experiment(name)
        print(f"Experiment created. ID: {experiment_id}")
    else:
        # If the experiment exists, set it
        mlflow.set_experiment(name)
        print(f"Experiment set to {name}")

4.Après avoir créé l'experiment. Nous allons entraîner notre modèle. Prenez le dataset wine de la librairie sklearn et utilisez un algorithme de la famille des arbres de décisions. Nous allons lors de l'entraînement de notre modèle logger les mesures.   

Pour cela nous voulons lancer un nouveau run dans notre experiment sur mlflow. Le run correspond à un entraînement du modèle.   

-Créer une variable run_name pour le nouveau run que vous voulez créer avec la date du jour, l'heure, la minute et la seconde dans le nom.   
-Trouvez comment logger les paramètres d'entraînement, les hyperparamètres et les performances du modèle (metrics) explicitement dans mlflow en lançant votre run de manière manuelle.   
-Ajoutez également votre modèle avec un nom distinctif grâce à la méthode adéquat, vous devrez réutiliser ce nom lors des prochains entraînements. Que voyez-vous dans l'ui de mlflow ?   


Vous pouvez utiliser `with` pour ne pas avoir besoin d'utiliser la fonction `end_run()`

In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
from datetime import datetime

# Load the wine dataset
wine = datasets.load_wine()
X = wine.data
y = wine.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a decision tree classifier
clf = tree.DecisionTreeClassifier()

# Train the classifier
clf = clf.fit(X_train, y_train)

# Predict the test set results
y_pred = clf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

# Create a run name with the current date and time
run_name = datetime.now().strftime("%Y%m%d-%H%M%S")

# Start a new run
with mlflow.start_run(run_name=run_name) as run:
    # Log the model parameters and metrics
    mlflow.log_param("criterion", clf.criterion)
    mlflow.log_param("splitter", clf.splitter)
    mlflow.log_metric("accuracy", accuracy)

    # Save the model with a distinctive name
    mlflow.sklearn.log_model(clf, "model")

print("Model trained and logged with run name: ", run_name)

Model trained and logged with run name:  20231115-104228


On peut utiliser `mlflow.active_run()` pour être sûr que le run est bien terminé.

In [7]:
mlflow.active_run() is None

True

Ecrivez un run sans utiliser un with et sans utilisez mlflow.end_run()

In [8]:
run = mlflow.start_run(run_name="My Run")
mlflow.log_param("param1", "value1")
mlflow.log_metric("metric1", 1.23)
mlflow.sklearn.log_model(clf, "model")

print("Run started with run_id: ", run.info.run_id)

Run started with run_id:  29817e4e842246de950e02f785397f8d


c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


La fonction `active_run()` nous retourne normalement ce run ci.

In [9]:
run = mlflow.active_run()
print(run)

<ActiveRun: >


Faisons bien attention de bien le fermer pour ne pas avoir de comportements exotiques ensuite.

Utilisez mlflow.end_run() et vérifiez avec active_run() qu'aucun run n'est retourné.

In [10]:
mlflow.end_run()
mlflow.active_run() is None

True

Il existe une autre manière de logger automatiquement des paramètres et des metrics à vous de la trouver.   
(Attention une fois activée cette fonction entrainera toujours un log automatique, veillez à la désactiver pour la suite)

In [16]:
import sklearn

mlflow.autolog(log_models=False, exclusive=True)
mlflow.sklearn.autolog(log_models=True)

clf = clf.fit(X_train, y_train)

2023/11/15 10:48:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils."
2023/11/15 10:48:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."


AssertionError: C:\Users\zacha\ApplicationDev\miniconda3\Lib\distutils\core.py

5.Changez un ou plusieurs hyperparamètres et relancez un entraînement avec le même nom de modèle. Rendez-vous dans l'onglet model de mlflow que constatez-vous ? 


In [17]:
# Create a decision tree classifier with different hyperparameters
clf = tree.DecisionTreeClassifier(max_depth=3)

# Train the classifier
clf = clf.fit(X_train, y_train)

# Log the model
with mlflow.start_run(run_name="My Run"):
    mlflow.sklearn.log_model(clf, "model")

c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Vous pouvez récupérer le dernier run avec la fonction last_active_run()

In [18]:
run = mlflow.last_active_run() 
print(run)

<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "7340acbfaa82495cbe1affc6d5a20022", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2023-11-15 09:58:54.453230", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.11.5", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"62041da82c354743b337efdad4295e45", '
                       

6.Entraîner 3,4 modèles avec des hyperparamètres différents.


In [19]:
from sklearn import tree

# Define different sets of hyperparameters
hyperparameters = [
    {"max_depth": 3, "min_samples_split": 2},
    {"max_depth": 5, "min_samples_split": 4},
    {"max_depth": None, "min_samples_split": 2},
    {"max_depth": 7, "min_samples_split": 10}
]

# Train a model for each set of hyperparameters
for i, params in enumerate(hyperparameters):
    # Create a decision tree classifier with the current hyperparameters
    clf = tree.DecisionTreeClassifier(max_depth=params["max_depth"], min_samples_split=params["min_samples_split"])

    # Train the classifier
    clf = clf.fit(X_train, y_train)

    # Log the model
    with mlflow.start_run(run_name=f"My Run {i+1}"):
        mlflow.log_params(params)
        mlflow.sklearn.log_model(clf, "model")

c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\envTP1\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils

## B. Versionning des données avec DVC

Pour chaque run le dataset utilisé est précisé dans mlflow. Mais les informations fournies sont assez pauvres. Pour améliorer ça nous allons utiliser l'outil dvc en combinaison avec mlflow.
dvc fonctionne de pair avec git.

Initialisez un repo git localement.

In [20]:
!git init

Initialized empty Git repository in C:/Users/zacha/Documents/CourS9/ADDE92Applications of Big Data/TP1/.git/


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>


Installez dvc avec pip

In [21]:
!pip install dvc

  Obtaining dependency information for dvc from https://files.pythonhosted.org/packages/e4/2b/2e04e73dbd9f87f5f4b3427e5053fe6c91f87e04c9028d704d91160f018a/dvc-3.29.0-py3-none-any.whl.metadata
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Obtaining dependency information for dpath<3,>=2.1.0 from https://files.pythonhosted.org/packages/84/c8/10c2d41a0958b76e777c07a521d64c871ab9022520babb3e08fa7eeb0810/dpath-2.1.6-py3-none-any.whl.metadata
  Obtaining dependency information for dvc-data<2.21.0,>=2.20.0 from https://files.pythonhosted.org/packages/d1/f3/0b62e3c2711ed3647032176f44482e8c19ba7925e66fb53fdfd7f6ca4b1f/dvc_data-2.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for dvc-render<1,>=0.3.1 from https://files.pythonhosted.org/packages/e4/52/675239b9451c327a462fc66a09f4c1c96a5b5d90faf566864e35e15a7791/dvc_render-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for dvc-studio-client<1,>=0.13.0 from https://files.pythonhosted.org/packages/54/3


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Initialisez un projet dvc.

In [22]:
!dvc init

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


Regardez ce qui a été créé avec un git status.

In [24]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   .dvc/.gitignore
	new file:   .dvc/config
	new file:   .dvcignore



Ajoutez les nouveaux fichiers dans le fichier .dvc : le .gitignore, le fichier config, et le .dvcignore.

In [25]:
!git commit -m "initialize repo"

[master 1d98e7b] initialize repo
 3 files changed, 6 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvcignore


Nous pouvons ajouter un stockage distant dans le cloud. Ici nous allons utiliser un stockage dans un dossier local pour les besoins du tp.

In [26]:
!dvc remote add -d dvc-remote /tmp/dvc-storage

Setting 'dvc-remote' as a default remote.


On peut regarder le contenu du fichier dvc config.

In [28]:
!type .dvc\config

[core]
    remote = dvc-remote
['remote "dvc-remote"']
    url = /tmp/dvc-storage


Nous constatons que l'url locale a bien été ajouté. Nous pouvons commiter ces changements à git.

In [29]:
!git commit .dvc/config -m "add remote storage"

[master 819acdb] add remote storage
 1 file changed, 4 insertions(+)


Nous allons créer un dossier data pour stocker notre dataset. 

In [30]:
!mkdir data

Télécharger notre dataset et le placer dans le dossier data.

In [31]:
import requests
import zipfile
import io
import pandas as pd


dataset_source_url = "https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip"

content = requests.get(dataset_source_url).content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

raw_data.to_csv(path_or_buf="data/hour.csv")

Affichons le contenu de data.

In [32]:
!dir data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:06    <DIR>          .
15/11/2023  11:05    <DIR>          ..
15/11/2023  11:06         1�161�688 hour.csv
               1 fichier(s)        1�161�688 octets
               2 R�p(s)  340�687�327�232 octets libres


Si nous voulons commencer à suivre les changements d'un fichier il nous suffit de l'ajouter via dvc.

In [33]:
!dvc add data/hour.csv


To track the changes with git, run:

	git add 'data\.gitignore' 'data\hour.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



Nous pouvons regarder de nouveau ce qu'il y a dans notre dossier.

In [34]:
!dir data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:06    <DIR>          .
15/11/2023  11:05    <DIR>          ..
15/11/2023  11:06                11 .gitignore
15/11/2023  11:06         1�161�688 hour.csv
15/11/2023  11:06                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  340�685�553�664 octets libres


Nous voyons un nouveau fichier .dvc.   
Si nous regardons à l'intérieur : 

In [38]:
!type data\hour.csv.dvc

outs:
- md5: 0cde6c4b0e77e95a1ff808ddc9da446a
  size: 1161688
  hash: md5
  path: hour.csv


On voit que le fichier contient des informations à propos de notre csv :  
- Un hash du fichier 
- l'algorithme de hashage utilisé
- la taille
- le chemin

Un gitignore a été créé par défaut, si on regarde à l'intérieur : 

In [39]:
!type data\.gitignore

/hour.csv


On peut voir que notre csv y est renseigné.

Maintenant ajoutons le nouveau fichier data/hour.csv.dvc et le fichier data/.gitignore

In [40]:
!git add data/.gitignore data/hour.csv.dvc

et commitons le

In [41]:
!git commit -m "add .dvc file to track hours.csv file"

[master aed3bce] add .dvc file to track hours.csv file
 2 files changed, 6 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 data/hour.csv.dvc


Une bonne idée est de créer un tag pour chaque version de notre dataset.

In [43]:
!git tag -a v1 -m "raw data"

Notre donnée est toujours sur notre dossier en local, maintenant nous devons l'envoyer sur notre stokage distant (qui pour rappel et en fait un autre dossier local). Pour ça nous utilisons la commande dvc push.

In [44]:
!dvc push

1 file pushed


On peut regarder dans notre "remote storage" ce que nous avons :

In [49]:
!dir /s \tmp\dvc-storage

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\tmp\dvc-storage

15/11/2023  11:08    <DIR>          .
15/11/2023  11:08    <DIR>          ..
15/11/2023  11:08    <DIR>          files
               0 fichier(s)                0 octets

 R�pertoire de c:\tmp\dvc-storage\files

15/11/2023  11:08    <DIR>          .
15/11/2023  11:08    <DIR>          ..
15/11/2023  11:08    <DIR>          md5
               0 fichier(s)                0 octets

 R�pertoire de c:\tmp\dvc-storage\files\md5

15/11/2023  11:08    <DIR>          .
15/11/2023  11:08    <DIR>          ..
15/11/2023  11:08    <DIR>          0c
               0 fichier(s)                0 octets

 R�pertoire de c:\tmp\dvc-storage\files\md5\0c

15/11/2023  11:08    <DIR>          .
15/11/2023  11:08    <DIR>          ..
15/11/2023  11:08         1�161�688 de6c4b0e77e95a1ff808ddc9da446a
               1 fichier(s)        1�161�688 octets

     Total des fichiers list�s�:
   

Nous pouvons voir que notre fichier est présent dans le dossier mais avec un nom différent, ce nom correspond au hash de la donnée du fichier.

Maintenant que nos données sont sauvegardées à distance, nous pouvons les supprimer localement. Sauf le fichier .dvc ! Car sinon vous perdrez le lien avec vos données.

In [50]:
!dir data
!rm -rf data/hour.csv

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:06    <DIR>          .
15/11/2023  11:05    <DIR>          ..
15/11/2023  11:06                11 .gitignore
15/11/2023  11:06         1�161�688 hour.csv
15/11/2023  11:06                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  340�681�932�800 octets libres


'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [51]:
!dir data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:06    <DIR>          .
15/11/2023  11:05    <DIR>          ..
15/11/2023  11:06                11 .gitignore
15/11/2023  11:06         1�161�688 hour.csv
15/11/2023  11:06                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  340�681�928�704 octets libres


Un autre emplacement où vos données résident est le dossier .dvc/cache

In [52]:
!dir .dvc\cache\files\md5\0a\1c63297d478edfdcc18433bb509cd5
!type .dvc\cache\files\md5\0a\1c63297d478edfdcc18433bb509cd5

Le fichier sp�cifi� est introuvable.
Le chemin d'acc�s sp�cifi� est introuvable.


Nous supprimons aussi les données à l'intérieur.

In [54]:
!rd /s /q .dvc\cache

Si nous voulons récupérer nos données localement, nous pouvons utiliser dvc pull pour récupérer les données.

In [55]:
!dvc pull

1 file fetched


Si nous regardons dans le dossier data :

In [56]:
!dir data

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:06    <DIR>          .
15/11/2023  11:05    <DIR>          ..
15/11/2023  11:06                11 .gitignore
15/11/2023  11:06         1�161�688 hour.csv
15/11/2023  11:06                96 hour.csv.dvc
               3 fichier(s)        1�161�795 octets
               2 R�p(s)  340�682�764�288 octets libres


Notre fichier est de retour.

Maintenant modifions nos données !

In [58]:
!dir data\hour.csv

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:06         1�161�688 hour.csv
               1 fichier(s)        1�161�688 octets
               0 R�p(s)  340�682�764�288 octets libres


In [61]:
# !sed -i '2,1001d' data/hour.csv
!powershell -Command "(Get-Content data\hour.csv | Select-Object -Skip 1000) | Set-Content data\hour.csv"

In [62]:
!dir data\hour.csv

 Le volume dans le lecteur C s'appelle OS
 Le num�ro de s�rie du volume est 506E-5B85

 R�pertoire de c:\Users\zacha\Documents\CourS9\ADDE92Applications of Big Data\TP1\data

15/11/2023  11:11         1�098�095 hour.csv
               1 fichier(s)        1�098�095 octets
               0 R�p(s)  340�682�588�160 octets libres


Et répétons les opérations précédantes : 

In [63]:
!dvc add data/hour.csv

⠋ Checking graph




To track the changes with git, run:

	git add 'data\hour.csv.dvc'

To enable auto staging, run:

	dvc config core.autostage true


In [64]:
!git add data/hour.csv.dvc

In [65]:
!git commit -m 'data: remove 1000 lines'

error: pathspec 'remove' did not match any file(s) known to git
error: pathspec '1000' did not match any file(s) known to git
error: pathspec 'lines'' did not match any file(s) known to git


In [78]:
!git tag -a v2 -m "removed 1000 lines"

In [67]:
!dvc push

1 file pushed


In [70]:
!rd /s /q .data\hour.csv

Le chemin d'acc�s sp�cifi� est introuvable.


In [71]:
!rd /s /q .dvc\cache

Nous pouvons maintenant regarder dans notre git log, et voir l'historique des modifications.

In [72]:
!git log

commit aed3bcef3ae76d32855bf41ebd6bb50dbe8420ce
Author: zacharyak30 <zachary.akakpo@efrei.net>
Date:   Wed Nov 15 11:07:43 2023 +0100

    add .dvc file to track hours.csv file

commit 819acdbfcbbb1cdae256de97aa9efe0a064cc872
Author: zacharyak30 <zachary.akakpo@efrei.net>
Date:   Wed Nov 15 11:05:50 2023 +0100

    add remote storage

commit 1d98e7badf3707887ee1e7fc6ce6d96ebe6a1a2e
Author: zacharyak30 <zachary.akakpo@efrei.net>
Date:   Wed Nov 15 11:04:50 2023 +0100

    initialize repo

commit ce791de11881aa46b8441aea859084585b02b952
Author: zacharyak30 <zachary.akakpo@efrei.net>
Date:   Wed Nov 15 11:01:48 2023 +0100

    starting


Pour accéder et extraire des versions spécifiques de nos données nous pouvons utiliser le package dvc en python. 

In [74]:
!pip freeze | findstr dvc

dvc==3.29.0
dvc-data==2.20.0
dvc-http==2.30.2
dvc-objects==1.2.0
dvc-render==0.6.0
dvc-studio-client==0.15.0
dvc-task==0.3.0


In [75]:
import dvc
import dvc.api

In [85]:
path = "data/hour.csv"  # The path to the file in your DVC repository
repo = "https://github.com/ZacharyAk30/TP1.git"  # The URL of your DVC repository
version = "v1"  # The version of your dataset

data_url = dvc.api.get_url(
    path=path, 
    repo=repo,
    rev=version
)


RevError: unknown Git revision 'v1'

In [82]:
MLFLOW_TRACKING_URI = "http://localhost:5000"
print(MLFLOW_TRACKING_URI)

http://localhost:5000


In [83]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
config_exp(name="ML_EXP_WITH_DVC")

Experiment created. ID: 188222493590758486


In [84]:
import pandas as pd

data = pd.read_csv(data_url, sep=",")
print(data.head())

NameError: name 'data_url' is not defined

In [86]:
from datetime import datetime, time

data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

NameError: name 'data' is not defined

In [ ]:
data.head()

Nous allons maintenant utiliser 2 mécanismes pour ajouter plus d'informations sur notre jeu de données dans MLFlow :    
-Grâce à dvc, nous avons maintenant des liens vers les différentes versions de notre jeu de données.   
-Nous pouvons l'utiliser en combinaison avec le module mlflow.data pour ajouter plus d'informations sur notre jeu de données.   

In [87]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [88]:
start_date = '2011-01-01 00:00:00'
end_date = '2011-01-28 23:00:00'
dataset = data.loc[start_date:end_date]

NameError: name 'data' is not defined

In [ ]:
print(dataset)

Dans la même cellule : 

1)
- Aller voir la documentation du module mlflow data et importer le bon objet pour les données pandas.  
- Créer un run avec la date, l'heure, etc. comme nom.    
- pour l'entrainement vous pouvez utiliser le chemin vers votre dataset lors de la création de votre dataframe.   

2) 
- logger le dataset avec la méthode appropriée.  
- logger également le chemin vers votre dataset.   
- logger la version du dataset utilisée.   

3)
- Créer un fichier texte et logger le en tant qu'artifact. Dans ce fichier vous pourrez indiquer la colonne qui a servi de target, les features numériques et les features catégorielles.

4)
- N'oubliez pas d'utiliser la fonction mlflow.end_run() si vous n'avez pas utilisez de with pour le run.  

In [ ]:
import mlflow
import pandas as pd
from datetime import datetime

# 1)
from mlflow.tracking import MlflowClient
client = MlflowClient()

# Create a run with date, time as name
run_name = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
run = client.create_run("0", run_name=run_name)

# Load your dataset
data_path = "path_to_your_dataset"
df = pd.read_csv(data_path)

# 2)
# Log the dataset
mlflow.log_param("data_path", data_path)
mlflow.log_param("data_version", "v1")  # replace with your data version

# 3)
# Create a text file and log it as an artifact
with open("info.txt", "w") as f:
    f.write("Target column: target\n")  # replace with your target column
    f.write("Numerical features: feature1, feature2\n")  # replace with your numerical features
    f.write("Categorical features: feature3, feature4\n")  # replace with your categorical features

mlflow.log_artifact("info.txt")

# 4)
# End the run
mlflow.end_run()

## C. Déploiement

7.Trouver comment transitionner un modèle en état staging et ensuite dans l'état production. 

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Transition model version 1 of model 'my_model' to 'Staging'
client.transition_model_version_stage(
  name="my_model",
  version=1,
  stage="Staging",
)

# Later, transition the same model version to 'Production'
client.transition_model_version_stage(
  name="my_model",
  version=1,
  stage="Production",
)

8.Créer une fonction qui récupère la version du modèle avec les meilleurs metrics d'entraînement et qui transitionne ce modèle dans l'état production.

In [ ]:
from mlflow.tracking import MlflowClient

def promote_best_model_to_production(model_name, metric_name):
    client = MlflowClient()

    # Get a list of all versions of the model
    versions = client.search_model_versions(f"name='{model_name}'")

    # Find the version with the best metric
    best_version = None
    best_metric = float('inf')
    for version in versions:
        run_id = version.run_id
        run = client.get_run(run_id)
        if metric_name in run.data.metrics:
            metric_value = run.data.metrics[metric_name]
            if metric_value < best_metric:
                best_metric = metric_value
                best_version = version.version

    if best_version is not None:
        # Transition the best version to 'Production'
        client.transition_model_version_stage(
            name=model_name,
            version=best_version,
            stage="Production",
        )
        print(f"Promoted version {best_version} of model {model_name} to 'Production'")
    else:
        print(f"No suitable version found for model {model_name}")

# Usage:
promote_best_model_to_production("my_model", "rmse")

9. Le serveur mlflow peut vous fournir des prédictions à partir des modèles enregistrés. Faites en sorte d'obtenir une prédiction de votre dernier modèle en requêtant le serveur mlflow. (Voir : https://mlflow.org/docs/latest/models.html#command-line-interface)

In [ ]:
import mlflow.pyfunc

# The path to the model in the MLflow server
model_uri = "runs:/<run_id>/model"

# Load the model
model = mlflow.pyfunc.load_model(model_uri)

# Create a sample input vector
input_vector = pd.DataFrame([1, 2, 3, 4, 5])

# Predict the output for the input vector
output_vector = model.predict(input_vector)

print(output_vector)

10.Créer un script à part qui pull le dernier modèle depuis le model registry. Plus tard vous pourrez utiliser ce script pour récupérer le modèle dans une api.

In [ ]:
from mlflow.tracking import MlflowClient

def get_latest_model(model_name):
    client = MlflowClient()
    model_version_details = client.get_latest_versions(model_name, stages=["Production"])
    if model_version_details:
        latest_version = model_version_details[0].version
        model_uri = f"models:/{model_name}/{latest_version}"
        model = mlflow.pyfunc.load_model(model_uri)
        return model
    else:
        print(f"No production model found for {model_name}")
        return None

# Usage:
model = get_latest_model("my_model")

## D. Entraînement d'un CNN et log des metrics dans MLFlow

Voir le notebook donnée par le formateur.